In [16]:
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [13]:
with open('data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)
    
    
with open('X.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    X = pickle.load(f)
with open('y.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    y = pickle.load(f)

In [22]:
count_vect = TfidfVectorizer()
X_transformed = count_vect.fit_transform(X)
X_transformed = X_transformed.A

In [18]:
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()

In [23]:
eclf1 = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(X_transformed, y)
print(eclf1.predict(X_transformed))

[1 1 1 ..., 0 0 0]


In [24]:
eclf2 = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3)],
        voting='soft')
eclf2 = eclf2.fit(X_transformed, y)
print(eclf2.predict(X_transformed))

[1 1 1 ..., 0 0 0]


In [28]:
eclf3 = VotingClassifier(estimators=[
       ('lr', clf1), ('rf', clf2), ('gnb', clf3)],
       voting='soft', weights=[2,1,1])
eclf3 = eclf3.fit(X_transformed, y)
print(eclf3.predict(X_transformed))

[1 1 1 ..., 0 0 0]
